In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import sys
import numpy as np
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import time
from datetime import datetime
from scipy import interpolate
import cartopy.crs as ccrs
import pyroms

DATA HALF HOUR VELOCITY FOR THE CASE OF NOVEMBER

Nb the grid is the same for the April and November so it's only created in the April Case

In [2]:
base_url = 'https://dap.ceda.ac.uk/thredds/dodsC/bodc/UOX220077/WINDS-M/1997/WINDS-M_SFC_1997.nc'
ds_complete_1997 = xr.open_dataset(base_url)

In [3]:
base_url = 'https://dap.ceda.ac.uk/thredds/dodsC/bodc/UOX220077/WINDS-M/1998/WINDS-M_SFC_1998.nc'
ds_complete_1998 = xr.open_dataset(base_url)

In [4]:
# Dropping unnecessary variables
dropping_variables = ds_complete_1997.drop_vars(['s_w','hc','s_rho',
                                 'theta_s','theta_b','Tcline','Vtransform','h','f','pm','pn','lon_rho','lat_rho',
                                  'angle', 'Cs_r','sc_r','Cs_w','lon_u','lon_v','lat_u', 'lat_v',
                                  'sc_w','mask_rho'])

# Clipping considering only the domain of interest for the thesis case study
clipped_domain_1997 = dropping_variables.sel(y_rho=slice(680, 986), x_rho=slice(150, 451),
                          y_u=slice(680, 986), x_u=slice(150, 451),
                          y_v=slice(680, 986), x_v=slice(150, 451))

In [5]:
# Dropping unnecessary variables
dropping_variables = ds_complete_1998.drop_vars(['s_w','hc','s_rho',
                                 'theta_s','theta_b','Tcline','Vtransform','h','f','pm','pn','lon_rho','lat_rho',
                                  'angle', 'Cs_r','sc_r','Cs_w','lon_u','lon_v','lat_u', 'lat_v',
                                  'sc_w','mask_rho'])

# Clipping considering only the domain of interest for the thesis case study
clipped_domain_1998 = dropping_variables.sel(y_rho=slice(680, 986), x_rho=slice(150, 451),
                          y_u=slice(680, 986), x_u=slice(150, 451),
                          y_v=slice(680, 986), x_v=slice(150, 451))

##### NOTE: Case of November as spawning month

Since starting from 1st November 1997, the 100th day after is the 8th February 1998, the 1998 dataset must be used too, with the same process done above.

To concatenate the two dataset with 61 + 39 days, it is possible to proceed in this way (it is very fast [9sec with rechunking, 15min without]):
- Chunk sizes
  
chunk_sizes = {'time_counter': 1, 'y_rho': 301, 'x_rho': 301, 'y_v': 301, 'x_v': 301, 'y_u': 301, 'x_u': 301}
- Rechunking the two datasets (the one with values of November and December and the one of January and February)
  
nov_dec_1997_rechunked = nov_dec_1997.chunk(chunk_sizes)

jan_feb_1998_rechunked = jan_feb_1998.chunk(chunk_sizes)
- Creating the rechunked 100 days dataset
  
days70_from_november_1997 = xr.concat([nov_dec_1997_rechunked, jan_feb_1998_rechunked], dim='time_counter')

Select the time interval Nov 1st 31 Dec

In [6]:
# Considering only 61 from Nov 1st
days_from_Nov_1997 = clipped_domain_1997.isel(time_counter=slice(14592,17520))

Select the time interval from Jan 1st to Feb 8th

In [7]:
# Considering only 39 from Jan 1st
days_from_Jan_1998 = clipped_domain_1998.isel(time_counter=slice(0,1873))

In [8]:
chunk_sizes = {'time_counter': 1, 'y_rho': 306, 'x_rho': 301, 'y_v': 306, 'x_v': 301,
               'y_u':306, 'x_u': 301}

In [9]:
days_from_Nov_1997_rechunked = days_from_Nov_1997.chunk(chunk_sizes)
days_from_Jan_1998_rechunked = days_from_Jan_1998.chunk(chunk_sizes)

In [10]:
days100_from_november_1997 = xr.concat([days_from_Nov_1997_rechunked, days_from_Jan_1998_rechunked], dim='time_counter')

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>


In [11]:
days100_from_november_1997_velocity_u = days100_from_november_1997.drop_vars(['nav_lon_v', 'nav_lat_v', 'v_surf', 'time'])
days100_from_november_1997_velocity_v = days100_from_november_1997.drop_vars(['nav_lon_u', 'nav_lat_u', 'u_surf', 'time'])

In [35]:
chunks = {'time_counter': 100, 'y_u': 306, 'x_u': 301}
days100_from_november_1997_velocity_u_chunked = days100_from_november_1997_velocity_u.chunk(chunks)
days100_from_november_1997_velocity_u_chunked.to_netcdf("/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_november_1997_velocity_u_OK.nc")

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></b

In [23]:
chunks = {'time_counter': 100, 'y_v': 306, 'x_v': 301}
days100_from_november_1997_velocity_v_chunked = days100_from_november_1997_velocity_v.chunk(chunks)
days100_from_november_1997_velocity_v_chunked.to_netcdf("/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_november_1997_velocity_v_OK.nc")

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>500 Internal Server Error</title></head><body><center><h1>500 Internal Server Error</h1></center><hr><center>nginx</center></body></html>
